This notebook is a wrap-up of all previsous notebooks. 
The goal is the generate 1 compiled data to feed into webapp.
Three datatables will inputed here:
(1)color matching database with 3175 palettes
(2)after Yolov3: 1534/1555 products with matching palette and recommendation 
(3)before Yolov3: 1880 products with matching palette and recommendation

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolor
import matplotlib.colors as mcolor
from ast import literal_eval
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import distance as distance

dir = os.getcwd()
home = os.path.dirname(os.path.dirname(dir)) ## 2 levels up of working directory dir

In [5]:
# import pallet file
palette_tuple = pd.read_csv(os.path.join(dir,r'palette_clean_hex.csv'), index_col = 0)

# reformat tuples after read_csv
# do the same for palette_tuple
palette_tuple['RGB1']=palette_tuple['RGB1'].apply(literal_eval)
palette_tuple['RGB2']=palette_tuple['RGB2'].apply(literal_eval)
palette_tuple['RGB3']=palette_tuple['RGB3'].apply(literal_eval)
palette_tuple['RGB4']=palette_tuple['RGB4'].apply(literal_eval)

print(palette_tuple.shape)
palette_tuple.head()

(3175, 8)


,RGB1,RGB2,RGB3,RGB4,hex1,hex2,hex3,hex4
Up Shore,"(2, 146, 183)","(26, 200, 219)","(140, 117, 106)","(222, 226, 236)",#0292b7,#1ac8db,#8c756a,#dee2ec
Thwarted Summer Shower,"(76, 82, 112)","(246, 82, 160)","(54, 238, 224)","(188, 236, 224)",#4c5270,#f652a0,#36eee0,#bcece0
Luscious Lemonade,"(225, 195, 64)","(76, 215, 208)","(164, 232, 224)","(248, 234, 140)",#e1c340,#4cd7d0,#a4e8e0,#f8ea8c
Rosettes and Cream,"(216, 167, 177)","(182, 226, 211)","(250, 232, 224)","(239, 124, 142)",#d8a7b1,#b6e2d3,#fae8e0,#ef7c8e
Berry Blues,"(78, 26, 61)","(207, 238, 209)","(255, 39, 104)","(5, 224, 233)",#4e1a3d,#cfeed1,#ff2768,#05e0e9


In [19]:
# import data with referecing palette and recommendation after yolo
data_w_yolo = pd.read_csv(os.path.join(dir, r'cropped_image_w_recommendation.csv'), index_col = 0)

data_w_yolo['match1_img'] = [data_w_yolo['index_image'][i] for i in data_w_yolo['match1']] 
data_w_yolo['match2_img'] = [data_w_yolo['index_image'][i] for i in data_w_yolo['match2']] 
data_w_yolo['match3_img'] = [data_w_yolo['index_image'][i] for i in data_w_yolo['match3']] 

data_w_yolo['match1_hex'] = [data_w_yolo['RGB_x_hex'][i] for i in data_w_yolo['match1']] 
data_w_yolo['match2_hex'] = [data_w_yolo['RGB_x_hex'][i] for i in data_w_yolo['match2']] 
data_w_yolo['match3_hex'] = [data_w_yolo['RGB_x_hex'][i] for i in data_w_yolo['match3']] 

# based off of 1534 catalog images (1555 fed into yolo and 1534 came out with detection at 0.05 threshold)
data_w_yolo.tail()

,dn_crop,dn_order,box_order,class_name,class_id,image,index_image,crop_width,crop_height,crop_area,...,match1,match2,match3,RGB_x_hex,match1_img,match2_img,match3_img,match1_hex,match2_hex,match3_hex
1529,img_1551_3014_264_Kitchenware.jpg.jpg,1551,3014,Kitchenware,264,Pillow_select/Page_9_60.jpg,Page_9_60,469,479,224651,...,1067,1178,428,#e9e4dc,Page_29_3,Page_31_54,Page_17_7,#9b8173,#606373,#87868b
1530,img_1552_3015_92_Stairs.jpg.jpg,1552,3015,Stairs,92,Pillow_select/Page_9_6.jpg,Page_9_6,408,445,181560,...,1236,741,106,#c3401b,Page_3_54,Page_23_37,Page_12_12,#158787,#ddccc1,#f4f5f1
1531,img_1553_3018_465_Furniture.jpg.jpg,1553,3018,Furniture,465,Pillow_select/Page_9_7.jpg,Page_9_7,441,373,164493,...,1022,950,250,#675450,Page_28_43,Page_26_5,Page_14_5,#101949,#dfb1cb,#a01822
1532,img_1554_3019_465_Furniture.jpg.jpg,1554,3019,Furniture,465,Pillow_select/Page_9_8.jpg,Page_9_8,406,311,126266,...,1224,1392,538,#e9e4de,Page_3_42,Page_7_22,Page_20_14,#fd8142,#afdae1,#6dc6d4
1533,img_1555_3021_465_Furniture.jpg.jpg,1555,3021,Furniture,465,Pillow_select/Page_9_9.jpg,Page_9_9,398,372,148056,...,670,1098,798,#6c6f7f,Page_22_29,Page_30_22,Page_24_32,#ccd4d4,#1d3874,#3c6599


In [11]:
# import coarse data with referecing palette and recommendation before yolo
# RGB were extracted directly from the entire image
data_no_yolo = pd.read_csv(os.path.join(dir, r'pillow_w_recommendation.csv'), index_col = 0)
# based off of 1880 catalog images (1555/1880 contains the same type/color products)
data_no_yolo.tail()

,Unnamed: 0,url,RGB_x,Proportion_x,RGB_y,Proportion_y,match1,match2,match3,palette_name,position_on_pltt,RGB_x_hex
index,,,,,,,,,,,,
Page_32_15,14,https://ak1.ostkcdn.com/images/products/is/ima...,"(254, 254, 254)",0.832538,"(217, 165, 77)",0.167462,1509,1468,1770,Morning Sun,0,#fefefe
Page_32_16,15,https://ak1.ostkcdn.com/images/products/126489...,"(43, 39, 34)",0.576574,"(253, 253, 251)",0.423426,577,1675,1818,Bear at Rest,1,#2b2722
Page_32_17,16,https://ak1.ostkcdn.com/images/products/226390...,"(56, 110, 128)",0.780294,"(254, 254, 254)",0.219706,462,810,419,Coffee Mates,0,#386e80
Page_32_18,17,https://ak1.ostkcdn.com/images/products/284589...,"(243, 242, 240)",0.677009,"(177, 163, 146)",0.322991,1516,1451,306,Baby Flower Clusters,1,#f3f2f0
Page_32_19,18,https://ak1.ostkcdn.com/images/products/284589...,"(241, 241, 240)",0.814369,"(169, 155, 139)",0.185631,1202,738,516,Cuddle Weather,2,#f1f1f0


In [22]:
# make a recommender table: recommender
r1 = data_w_yolo[['dn_crop', 'image','palette_name','position_on_pltt','index_image','match1_img','match2_img','match3_img',
                  'RGB_x_hex','match1_hex','match2_hex','match3_hex']].copy()
r1.index = data_w_yolo['index_image']
r2 = data_no_yolo[['url','RGB_x_hex']].rename(index=str, columns={"RGB_x_hex": "old_hex"})

r1 = r1.rename(index=str, columns={'RGB_x_hex': 'user_hex'})
recommender = r1.join(r2, how='left')
print(recommender.shape)
recommender.head()

(1534, 14)


,dn_crop,image,palette_name,position_on_pltt,index_image,match1_img,match2_img,match3_img,user_hex,match1_hex,match2_hex,match3_hex,url,old_hex
index_image,,,,,,,,,,,,,,
Page_10_11,img_1_1_192_Food.jpg.jpg,Pillow_select/Page_10_11.jpg,Newborn Bundle,2,Page_10_11,Page_18_31,Page_16_29,Page_5_19,#aea294,#c44d73,#107a9a,#a7a92a,https://ak1.ostkcdn.com/images/products/185365...,#fcfbfa
Page_10_12,img_2_4_432_Clothing.jpg.jpg,Pillow_select/Page_10_12.jpg,Natural Flush,2,Page_10_12,Page_25_17,Page_3_6,Page_21_27,#d5524f,#51504f,#e7bfb7,#d37d7a,https://ak1.ostkcdn.com/images/products/861039...,#fefefe
Page_10_13,img_3_5_365_Picture frame.jpg.jpg,Pillow_select/Page_10_13.jpg,Seaweed Sushi,3,Page_10_13,Page_31_47,Page_18_11,Page_14_19,#d7d7db,#bbc6cd,#39625d,#232f3e,https://ak1.ostkcdn.com/images/products/235857...,#fcfcfc
Page_10_14,img_4_6_465_Furniture.jpg.jpg,Pillow_select/Page_10_14.jpg,Secret Romance,1,Page_10_14,Page_24_58,Page_32_15,Page_4_57,#121212,#bd0816,#d9a54d,#ede8d8,https://ak1.ostkcdn.com/images/products/901271...,#fdfdfd
Page_10_15,img_5_9_446_Book.jpg.jpg,Pillow_select/Page_10_15.jpg,Living Details,1,Page_10_15,Page_18_22,Page_15_11,Page_7_41,#e9e8e6,#6e544f,#c5b59f,#747472,https://ak1.ostkcdn.com/images/products/955129...,#ebeae8


In [23]:
export_csv = recommender.to_csv (os.path.join(dir,r'fina_recommendation.csv'), index = True, header=True)

So! It's freaking working! 
Use first row for example: search for palette_name "Newborn Bundle" in https://www.canva.com/colors/color-palettes/.
Then, look at how "user_hex", "match1-3_hex" match with the palette!
I checked the 4 actual images and they are matching with the palette too!
Now I'm heading back to my work app development.